<a href="https://colab.research.google.com/github/ferbator/data-engineering/blob/main/M33071_%D0%9A%D0%B0%D1%80%D0%B5%D0%BF%D0%B8%D0%BD_%22lab4%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вершину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [ ]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы) для расчёта энтропии (уже есть), для разделения дерева (используйте `pandas`), для подсчёта функционала качества $IG$, для выбора наилучшего разделения (с учетом признакоd и порогов), для проверки критерия остановки.

Для набора данных `iris` реализуйте алгоритм и минимум три из разными критерия остановки из перечисленных ниже:
* максимальной глубины дерева = 5
* минимального числа объектов в листе = 5
* максимальное количество листьев в дереве = 5
* purity (остановка, если все объекты в листе относятся к одному классу)

Реализуйте функцию `predict` (на вход функции подаётся датафрейм с объектами)

Оцените точность каждой модели с помощью метрики доля правильных ответов (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [ ]:
class Node:
    def __init__(self, feature=None, t=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.t = t
        self.value = value
        self.left = left
        self.right = right

In [ ]:
class DecisionTree:
    def __init__(self, max_depth=5, min_leaf=5):
        self.max_depth = max_depth
        self.min_leaf = min_leaf
        self.tree = None

    def fit(self, X: np.ndarray, y: np.ndarray, stop_criteria: str):
        self.tree = self.grow_tree(X, y, stop_criteria)

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self.travers_tree(x, self.tree) for x in X])

    def most_common(self, y: np.ndarray) -> np.float64:
        labels = np.unique(y)
        count = [list(y).count(i) for i in labels]
        return labels[np.argmax(count)]

    def best_split(self, X: np.ndarray, y: np.ndarray):
        best_feature, best_t, best_gain = None, None, -1
        for i in range(X.shape[1]):
            thresholds = np.unique(X[:, i])
            for threshold in thresholds:
                gain = self.information_gain(X[:, i], y, threshold)
                if gain > best_gain:
                    best_gain, best_feature, best_t = gain, i, threshold
        return best_feature, best_t

    def information_gain(self, X_column, y, threshold):
        if len(np.unique(y)) == 1:
            return 0
        n, parent = len(y), entropy(y)
        left_indexes, right_indexes = np.argwhere(X_column <= threshold).flatten(), np.argwhere(X_column > threshold).flatten()
        e_l, n_l, e_r, n_r = entropy(y[left_indexes]), len(left_indexes), entropy(y[right_indexes]), len(right_indexes)
        child = (n_l / n) * e_l + (n_r / n) * e_r
        return parent - child

    def grow_tree(self, X, y, criteria, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        def check():
            if criteria == 'max_depth':
                return n_labels == 1 or depth >= self.max_depth
            elif criteria == 'min_leaf':
                return n_labels == 1 or len(y) < self.min_leaf
            elif criteria == 'purity':
                return all(x == y[0] for x in y)
        if check():
            return Node(value=self.most_common(y))
        best_feature, best_t = self.best_split(X, y)
        left_indexes = np.argwhere(X[:, best_feature] <= best_t).flatten()
        right_indexes = np.argwhere(X[:, best_feature] > best_t).flatten()
        left = self.grow_tree(X[left_indexes, :], y[left_indexes], criteria, depth + 1)
        right = self.grow_tree(X[right_indexes, :], y[right_indexes], criteria, depth + 1)
        return Node(best_feature, best_t, left, right)

    def travers_tree(self, x: np.ndarray, tree: Node):
        if tree.value is not None:
            return tree.value
        if x[tree.feature] <= tree.t:
            return self.travers_tree(x, tree.left)
        return self.travers_tree(x, tree.right)

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)
tree1 = DecisionTree(max_depth=5)
tree2 = DecisionTree(min_leaf=5)
tree3 = DecisionTree()

tree1.fit(X_train, y_train, 'max_depth')
tree2.fit(X_train, y_train, 'min_leaf')
tree3.fit(X_train, y_train, 'purity')

predicted1 = tree1.predict(X_test)
predicted2 = tree2.predict(X_test)
predicted3 = tree3.predict(X_test)

print("Accuracy score with max_depth", accuracy_score(y_test, predicted1))
print("Accuracy score with min_leaf", accuracy_score(y_test, predicted2))
print("Accuracy score with purity", accuracy_score(y_test, predicted3))

Accuracy score with max_depth 0.8947368421052632
Accuracy score with min_leaf 0.9210526315789473
Accuracy score with purity 0.8947368421052632


AxisError: ignored

##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

churn = pd.read_csv('drive/MyDrive/ML/data/churn.csv')

churn.head()

Mounted at /content/drive


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
churn = churn.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

churn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [ ]:
churn_encoded = pd.get_dummies(churn, columns=['Geography', 'Gender'])

In [ ]:
churn_encoded

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,1,0,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,1,0,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,1,1,0,0,1,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0,0,1


In [ ]:
from collections import Counter
from sklearn.base import BaseEstimator
from sklearn.tree import DecisionTreeClassifier

In [ ]:
class RandomForest(BaseEstimator):
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=10, criterion='entropy', max_leaf_nodes=20):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criterion = criterion
        self.max_leaf_nodes = max_leaf_nodes
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for i in range(self.n_trees):
            tree = DecisionTreeClassifier(max_depth=self.max_depth,
                                      min_samples_split=self.min_samples_split,
                                      criterion=self.criterion,
                                      max_leaf_nodes=self.max_leaf_nodes, max_features='sqrt', class_weight='balanced')
            n_samples = len(X)
            indexes = np.random.choice(n_samples, n_samples, replace=True)
            X_ = X.iloc[indexes, :]
            y_ = y.iloc[indexes]
            tree.fit(X_, y_)
            self.trees.append(tree)

    def predict(self, X):
        predictions = np.swapaxes(np.array([tree.predict(X) for tree in self.trees]), 0, 1)
        y_predicion = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in predictions]
        return np.array(y_predicion)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
np.random.seed(0)
X = churn_encoded.drop(columns='Exited')
y = churn_encoded['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=2)

forest = RandomForest()

GSCV = GridSearchCV(forest, {
    'n_trees': [50, 100, 150],
    'min_samples_split': [2, 5, 8],
    'max_depth': [10, 12, 14]},
     scoring='accuracy', n_jobs=-1)

GSCV.fit(X_train, y_train)

b_e = GSCV.best_estimator_
print(b_e)

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
b_e.fit(X_train, y_train)
accuracy_score(y_test, b_e.predict(X_test))

In [ ]:
important_parameters = []

for tree in GSCV.best_estimator_.trees:
    important_parameters.append(tree.feature_importances_)

important_parameters = np.array(important_parameters)
importance = pd.DataFrame(data= important_parameters, columns=X.columns)

importance = importance.mean().sort_values(ascending=False)

importance.head()

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, b_e.predict(X_test)))